# Forest Fire Prediction and Simulation - Data Analysis

This notebook provides exploratory data analysis and visualization tools for the forest fire prediction project.

## Contents
1. Data Loading and Inspection
2. Environmental Data Analysis
3. Historical Fire Pattern Analysis
4. Correlation Analysis
5. Data Quality Assessment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import rasterio
import rasterio.plot
from rasterio.plot import show
import folium
from folium import plugins
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append('../src')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 1. Data Loading and Inspection

In [ ]:
def load_raster_data(file_path, band=1):
    """Load raster data and return array with metadata."""
    try:
        with rasterio.open(file_path) as src:
            data = src.read(band)
            profile = src.profile
            bounds = src.bounds
            crs = src.crs
            
        return {
            'data': data,
            'profile': profile,
            'bounds': bounds,
            'crs': crs,
            'shape': data.shape,
            'dtype': data.dtype
        }
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def inspect_raster(raster_info, name):
    """Print raster inspection summary."""
    if raster_info is None:
        print(f"{name}: Failed to load")
        return
        
    data = raster_info['data']
    print(f"\n{name} Raster Information:")
    print(f"  Shape: {raster_info['shape']}")
    print(f"  Data type: {raster_info['dtype']}")
    print(f"  CRS: {raster_info['crs']}")
    print(f"  Bounds: {raster_info['bounds']}")
    print(f"  Min value: {np.nanmin(data):.4f}")
    print(f"  Max value: {np.nanmax(data):.4f}")
    print(f"  Mean value: {np.nanmean(data):.4f}")
    print(f"  NoData pixels: {np.sum(np.isnan(data))} ({np.sum(np.isnan(data))/data.size*100:.2f}%)")

# Example usage (modify paths to your actual data)
data_dir = "../data/raw"

# Define sample data files (replace with your actual files)
sample_files = {
    "Temperature": os.path.join(data_dir, "temperature.tif"),
    "Humidity": os.path.join(data_dir, "humidity.tif"),
    "DEM": os.path.join(data_dir, "dem.tif"),
    "Land Cover": os.path.join(data_dir, "landcover.tif")
}

# Load and inspect data (only if files exist)
raster_data = {}
for name, file_path in sample_files.items():
    if os.path.exists(file_path):
        raster_data[name] = load_raster_data(file_path)
        inspect_raster(raster_data[name], name)
    else:
        print(f"File not found: {file_path}")
        
print(f"\nLoaded {len(raster_data)} raster datasets.")

## 2. Environmental Data Visualization

In [ ]:
def plot_raster_data(raster_info, title, cmap='viridis', figsize=(10, 8)):
    """Plot raster data with statistics."""
    if raster_info is None:
        print(f"Cannot plot {title}: No data loaded")
        return
        
    data = raster_info['data']
    
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    
    # Raster plot
    im = axes[0].imshow(data, cmap=cmap, aspect='equal')
    axes[0].set_title(f'{title} - Spatial Distribution')
    axes[0].set_xlabel('X (pixels)')
    axes[0].set_ylabel('Y (pixels)')
    plt.colorbar(im, ax=axes[0])
    
    # Histogram
    valid_data = data[~np.isnan(data)]
    axes[1].hist(valid_data, bins=50, alpha=0.7, edgecolor='black')
    axes[1].set_title(f'{title} - Value Distribution')
    axes[1].set_xlabel('Value')
    axes[1].set_ylabel('Frequency')
    axes[1].axvline(np.mean(valid_data), color='red', linestyle='--', label=f'Mean: {np.mean(valid_data):.2f}')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()

# Plot loaded raster data
cmaps = {
    "Temperature": "RdYlBu_r",
    "Humidity": "Blues",
    "DEM": "terrain",
    "Land Cover": "tab20"
}

for name, data in raster_data.items():
    cmap = cmaps.get(name, 'viridis')
    plot_raster_data(data, name, cmap=cmap)

## 3. Generate Sample Data for Demonstration

In [ ]:
def generate_sample_data(shape=(500, 500)):
    """Generate sample environmental data for demonstration."""
    np.random.seed(42)
    
    # Create coordinate grids
    y, x = np.ogrid[:shape[0], :shape[1]]
    
    # Temperature (decreases with elevation)
    elevation = 100 + 1500 * np.sin(y/100) * np.cos(x/150) + 200 * np.random.random(shape)
    temperature = 35 - elevation/100 + 5 * np.random.random(shape)
    
    # Humidity (varies with location)
    humidity = 40 + 30 * np.sin(y/80) + 10 * np.random.random(shape)
    humidity = np.clip(humidity, 0, 100)
    
    # Wind speed
    wind_speed = 5 + 10 * np.random.random(shape)
    
    # Fuel load (land cover based)
    fuel_load = np.random.choice([0.1, 0.3, 0.6, 0.8, 0.9], size=shape, p=[0.1, 0.2, 0.3, 0.3, 0.1])
    
    # Historical fires (sparse)
    fire_history = np.random.choice([0, 1], size=shape, p=[0.98, 0.02])
    
    return {
        'temperature': temperature,
        'humidity': humidity,
        'elevation': elevation,
        'wind_speed': wind_speed,
        'fuel_load': fuel_load,
        'fire_history': fire_history
    }

# Generate sample data
sample_data = generate_sample_data()

print("Generated sample environmental data:")
for name, data in sample_data.items():
    print(f"  {name}: shape {data.shape}, range [{data.min():.2f}, {data.max():.2f}]")

## 4. Correlation Analysis

In [ ]:
def analyze_correlations(data_dict, target_var='fire_history'):
    """Analyze correlations between environmental variables and fire occurrence."""
    
    # Flatten arrays and create DataFrame
    df_data = {}
    for name, data in data_dict.items():
        df_data[name] = data.flatten()
    
    df = pd.DataFrame(df_data)
    
    # Remove NaN values
    df = df.dropna()
    
    # Calculate correlation matrix
    corr_matrix = df.corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0,
                square=True, fmt='.3f', cbar_kws={'label': 'Correlation Coefficient'})
    plt.title('Environmental Variables Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    # Print correlations with target variable
    if target_var in df.columns:
        target_corr = corr_matrix[target_var].drop(target_var).sort_values(key=abs, ascending=False)
        print(f"\nCorrelations with {target_var}:")
        print("-" * 40)
        for var, corr in target_corr.items():
            print(f"{var:15s}: {corr:6.3f}")
    
    return df, corr_matrix

# Analyze sample data correlations
df, corr_matrix = analyze_correlations(sample_data)

## 5. Fire Risk Analysis

In [ ]:
def analyze_fire_risk_factors(df):
    """Analyze environmental conditions in fire vs non-fire areas."""
    
    if 'fire_history' not in df.columns:
        print("No fire history data available for analysis")
        return
    
    # Separate fire and non-fire areas
    fire_areas = df[df['fire_history'] == 1]
    no_fire_areas = df[df['fire_history'] == 0]
    
    print(f"Fire areas: {len(fire_areas)} pixels ({len(fire_areas)/len(df)*100:.2f}%)")
    print(f"No-fire areas: {len(no_fire_areas)} pixels ({len(no_fire_areas)/len(df)*100:.2f}%)")
    
    # Environmental variables to analyze
    env_vars = ['temperature', 'humidity', 'elevation', 'wind_speed', 'fuel_load']
    
    # Create comparison plots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    for i, var in enumerate(env_vars):
        if var not in df.columns:
            continue
            
        # Box plot comparison
        data_to_plot = [no_fire_areas[var], fire_areas[var]]
        axes[i].boxplot(data_to_plot, labels=['No Fire', 'Fire'])
        axes[i].set_title(f'{var.title()} Distribution')
        axes[i].set_ylabel(var.title())
        
        # Add mean lines
        axes[i].axhline(no_fire_areas[var].mean(), color='blue', linestyle='--', alpha=0.7)
        axes[i].axhline(fire_areas[var].mean(), color='red', linestyle='--', alpha=0.7)
    
    # Remove empty subplot
    if len(env_vars) < len(axes):
        axes[-1].remove()
    
    plt.tight_layout()
    plt.show()
    
    # Statistical comparison
    from scipy import stats
    
    print("\nStatistical Comparison (Fire vs No-Fire):")
    print("=" * 50)
    print(f"{'Variable':<15} {'Fire Mean':<12} {'No-Fire Mean':<15} {'P-value':<10} {'Significant':<12}")
    print("-" * 70)
    
    for var in env_vars:
        if var not in df.columns:
            continue
            
        fire_mean = fire_areas[var].mean()
        no_fire_mean = no_fire_areas[var].mean()
        
        # T-test
        statistic, p_value = stats.ttest_ind(fire_areas[var], no_fire_areas[var])
        significant = "Yes" if p_value < 0.05 else "No"
        
        print(f"{var:<15} {fire_mean:<12.3f} {no_fire_mean:<15.3f} {p_value:<10.3e} {significant:<12}")

# Analyze fire risk factors
analyze_fire_risk_factors(df)

## 6. Interactive Map Visualization

In [ ]:
def create_interactive_map(data_dict, center_lat=40.0, center_lon=-120.0):
    """Create interactive map with environmental layers."""
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    # Add layer control
    folium.plugins.Fullscreen().add_to(m)
    
    # Note: In a real scenario, you would add raster overlays here
    # For demonstration, we'll add sample markers
    
    # Add sample fire locations
    if 'fire_history' in data_dict:
        fire_data = data_dict['fire_history']
        fire_locations = np.where(fire_data == 1)
        
        # Sample some fire locations to avoid overcrowding
        n_samples = min(100, len(fire_locations[0]))
        indices = np.random.choice(len(fire_locations[0]), n_samples, replace=False)
        
        for i in indices:
            lat = center_lat + (fire_locations[0][i] - fire_data.shape[0]/2) * 0.001
            lon = center_lon + (fire_locations[1][i] - fire_data.shape[1]/2) * 0.001
            
            folium.CircleMarker(
                location=[lat, lon],
                radius=3,
                popup='Historical Fire',
                color='red',
                fillColor='red',
                fillOpacity=0.7
            ).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; 
                top: 10px; right: 10px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <h4>Legend</h4>
    <i class="fa fa-circle" style="color:red"></i> Historical Fire Locations<br>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Create interactive map
interactive_map = create_interactive_map(sample_data)
interactive_map

## 7. Data Quality Assessment

In [ ]:
def assess_data_quality(data_dict):
    """Assess data quality and completeness."""
    
    print("DATA QUALITY ASSESSMENT")
    print("=" * 50)
    
    quality_report = {}
    
    for name, data in data_dict.items():
        total_pixels = data.size
        valid_pixels = np.sum(~np.isnan(data))
        missing_pixels = total_pixels - valid_pixels
        missing_percentage = (missing_pixels / total_pixels) * 100
        
        # Check for outliers (values beyond 3 standard deviations)
        valid_data = data[~np.isnan(data)]
        if len(valid_data) > 0:
            mean_val = np.mean(valid_data)
            std_val = np.std(valid_data)
            outliers = np.sum(np.abs(valid_data - mean_val) > 3 * std_val)
            outlier_percentage = (outliers / len(valid_data)) * 100
        else:
            outliers = 0
            outlier_percentage = 0
        
        quality_report[name] = {
            'total_pixels': total_pixels,
            'valid_pixels': valid_pixels,
            'missing_pixels': missing_pixels,
            'missing_percentage': missing_percentage,
            'outliers': outliers,
            'outlier_percentage': outlier_percentage
        }
        
        print(f"\n{name.upper()}:")
        print(f"  Total pixels: {total_pixels:,}")
        print(f"  Valid pixels: {valid_pixels:,} ({100-missing_percentage:.2f}%)")
        print(f"  Missing pixels: {missing_pixels:,} ({missing_percentage:.2f}%)")
        print(f"  Outliers (>3σ): {outliers:,} ({outlier_percentage:.2f}%)")
        
        # Quality assessment
        if missing_percentage < 5:
            quality = "Excellent"
        elif missing_percentage < 15:
            quality = "Good"
        elif missing_percentage < 30:
            quality = "Fair"
        else:
            quality = "Poor"
        
        print(f"  Data Quality: {quality}")
    
    # Summary plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Missing data percentage
    names = list(quality_report.keys())
    missing_pcts = [quality_report[name]['missing_percentage'] for name in names]
    
    bars1 = ax1.bar(names, missing_pcts, color='lightcoral')
    ax1.set_title('Missing Data Percentage by Variable')
    ax1.set_ylabel('Missing Data (%)')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, pct in zip(bars1, missing_pcts):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{pct:.1f}%', ha='center', va='bottom')
    
    # Outlier percentage
    outlier_pcts = [quality_report[name]['outlier_percentage'] for name in names]
    
    bars2 = ax2.bar(names, outlier_pcts, color='lightskyblue')
    ax2.set_title('Outlier Percentage by Variable')
    ax2.set_ylabel('Outliers (%)')
    ax2.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, pct in zip(bars2, outlier_pcts):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{pct:.1f}%', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    return quality_report

# Assess data quality
quality_report = assess_data_quality(sample_data)

## Summary

This notebook provides comprehensive analysis tools for the forest fire prediction project:

1. **Data Loading and Inspection**: Functions to load and examine raster datasets
2. **Visualization**: Tools for plotting environmental data and distributions
3. **Correlation Analysis**: Methods to identify relationships between variables
4. **Fire Risk Analysis**: Comparison of environmental conditions in fire vs non-fire areas
5. **Interactive Mapping**: Folium-based maps for spatial visualization
6. **Data Quality Assessment**: Evaluation of data completeness and quality

**Next Steps:**
- Replace sample data with your actual environmental datasets
- Modify file paths in the configuration
- Run the main pipeline using the insights gained from this analysis

**Note:** This notebook uses sample data for demonstration. Update the file paths to use your actual data files.